In [ ]:
from keras import metrics
from model import neural_voice_judgment_model
import idx2numpy
from os.path import join
import numpy as np
from sample_yomikomi import sample_all_data
from keras.callbacks import EarlyStopping
import tensorflow as tf
from twenty_times_more_data import twenty_times_more_data
import matplotlib.pyplot as plt
import numpy as np
from keras.models import Model
import keras.backend as K

In [ ]:
test_label_data = idx2numpy.convert_from_file(join("..","dataset","test_label.idx"))
test_image_data = idx2numpy.convert_from_file(join("..","dataset","test_image.idx"))
input_output_train_label_data = idx2numpy.convert_from_file(join("..","dataset","train_label.idx"))
input_output_train_image_data = idx2numpy.convert_from_file(join("..","dataset","train_image.idx"))
input_output_validation_label_data = idx2numpy.convert_from_file(join("..","dataset","validation_label.idx"))
input_output_validation_image_data = idx2numpy.convert_from_file(join("..","dataset","validation_image.idx"))

In [ ]:
model = neural_voice_judgment_model()
model.build((None,128,128,1))
model.summary()

In [ ]:
def total_acc(y_true: np.ndarray, y_pred: np.ndarray):
    pred = K.cast(K.greater_equal(y_pred, 0.5), "float")
    flag = K.cast(K.equal(y_true, pred), "float")
    return K.prod(flag, axis=-1)

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=[
        "binary_accuracy",
        total_acc,
    ],
)

In [ ]:
train_label_dataset = tf.data.Dataset.from_tensor_slices(input_output_train_label_data)
train_image_dataset = tf.data.Dataset.from_tensor_slices(input_output_train_image_data)
train_lump_dataset = tf.data.Dataset.zip((train_image_dataset,train_label_dataset))
callback = EarlyStopping(patience=10)
flat_map_dataset = train_lump_dataset.flat_map(twenty_times_more_data)

In [ ]:
fit_data = model.fit(
    flat_map_dataset,
    epochs=10,
    steps_per_epoch=3125,
    callbacks=[callback],
    validation_data=(input_output_validation_image_data,input_output_validation_label_data),
    validation_steps=32,
)

In [ ]:
plt.plot(fit_data.history["loss"])
plt.plot(fit_data.history["val_loss"])

In [ ]:
plt.plot(fit_data.history["binary_accuracy"])
plt.plot(fit_data.history["val_binary_accuracy"])

In [ ]:
plt.plot(fit_data.history["total_acc"])
plt.plot(fit_data.history["val_total_acc"])

In [ ]:
model.evaluate(x=test_image_data,y=test_label_data)

In [ ]:
model.save(join("trained_model","fifth.tf"))